# Threshold Joye-Libert Secure Aggregation

In [1]:
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS
from ftsa.protocols.buildingblocks.VectorEncoding import VES
import random

t = 412
threshold = 7
nclients = 10
drops = 3
keysize = 2048 
inputsize = 16
dimension = 10

users = list(range(1, nclients+1))
alive = list(range(1, nclients-drops+1))
dropped = list(range(nclients-drops+1,nclients+1))


VE = VES(keysize // 2, nclients, inputsize, dimension)
TJL = TJLS(nclients, threshold, VE)

print("users: ", users)
print("alive: ", alive)
print("dropped: ", dropped)

users:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alive:  [1, 2, 3, 4, 5, 6, 7]
dropped:  [8, 9, 10]


### Generate JL keys (**Setup**)

In [2]:
# generate keys
pp, skey, ukey = TJL.Setup(keysize)

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=14439...04021, H(x)=0x323b6f6)>
 - Aggregator Key: <ServerKey -0x4415d9f>
 - User Keys: {0: <UserKey 0x12829774>, 1: <UserKey 0x13fe5685>, 2: <UserKey 0x1c14bead>, 3: <UserKey 0x1ed5b656>, 4: <UserKey 0x15f15b5c>, 5: <UserKey 0x1a0696e8>, 6: <UserKey 0x1112f218>, 7: <UserKey 0x1d7875b2>, 8: <UserKey 0xa141d054>, 9: <UserKey 0x1a3e838b>}


### Share the JL user key with all other users (**SKShare**)


In [3]:

# share secret key of each user
shares = {}
for u in users: 
    shares[u] = TJL.SKShare(ukey[u-1],threshold, users)

# distribute the shares
from collections import defaultdict
usersshares = defaultdict(dict)
for v in users:
    for u in users: 
        usersshares[u][v] = shares[v][u-1]

### Protect inputs of alive users with the users' keys (**Protect**)

In [4]:
# generate $nclients random vectors
L = []
for j in range(nclients):
    l =[]
    for i in range(dimension):
        l.append(random.randint(0,1000))
    L.append(l)
# encrypt all vectors
E=[]
for u in alive:
    e = TJL.Protect(pp, ukey[u-1], t, L[u-1])
    E.append(e)
    print("user {}:".format(u), L[u-1], e)

user 1: [87, 54, 381, 174, 855, 296, 904, 492, 654, 923] [<EncryptedNumber 18922...10064>]
user 2: [793, 779, 564, 758, 653, 473, 543, 379, 462, 801] [<EncryptedNumber 12039...14519>]
user 3: [146, 253, 458, 734, 824, 318, 659, 731, 120, 368] [<EncryptedNumber 15255...26257>]
user 4: [122, 553, 662, 796, 327, 312, 237, 580, 567, 200] [<EncryptedNumber 77872...01745>]
user 5: [793, 145, 868, 931, 439, 64, 113, 705, 775, 976] [<EncryptedNumber 80153...01340>]
user 6: [233, 656, 331, 527, 832, 644, 887, 166, 932, 993] [<EncryptedNumber 66013...09853>]
user 7: [455, 533, 530, 949, 764, 312, 874, 699, 634, 626] [<EncryptedNumber 36859...04841>]


### Protect the zero-value with the key shares of the dropped users  (**ShareProtect**)

In [5]:
if dropped != []:
    yzeroshare ={}
    for u in alive:
        dropped_users_shares = []
        for v in dropped:
            dropped_users_shares.append(usersshares[u][v])
        yzeroshare[u] = TJL.ShareProtect(pp, dropped_users_shares,t)

yzeroshare

{1: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e3070>],
 2: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e3190>],
 3: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e38b0>],
 4: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e39a0>],
 5: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e0b80>],
 6: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e25c0>],
 7: [<ftsa.protocols.buildingblocks.IntegerSS.IShare at 0x7faea64e26b0>]}

### Combine shares of the protected zero-value (**ShareCombine**)

In [6]:
if dropped != []:
    yzero = TJL.ShareCombine(pp, list(yzeroshare.values()), threshold)
else:
    yzero = None

yzero

[<EncryptedNumber 12779...38970>]

### Aggregate all user protected inputs and the protected zero-value of dropped users (**Agg**)

In [7]:
S = TJL.Agg(pp, skey, t, E, yzero)


summ=L[0]
from operator import add
for l in L[1:nclients-drops]:
    summ = list(map(add, summ, l))

print(S)
print(summ)
# check if the result is correct
print("Verify: ", S == summ)

[2629, 2973, 3794, 4869, 4694, 2419, 4217, 3752, 4144, 4887]
[2629, 2973, 3794, 4869, 4694, 2419, 4217, 3752, 4144, 4887]
Verify:  True
